In [1]:
import requests
import urllib3
import pandas as pd
from pyhive import presto
import os
import sys
import pymysql
from sqlalchemy import create_engine
from pyhive import presto
import urllib3
import requests
import json
from shapely.geometry import shape, Point
import warnings; warnings.filterwarnings('ignore')
import qgrid
import subprocess

In [2]:
#get credentials (login and password)
os.chdir('C:/Users/maxim.buldakov/Desktop/Private')
ppp = pd.read_csv('ppp.txt')

def query_presto(q):
    presto_session = requests.Session()
    presto_session.verify = 'C:/Users/maxim.buldakov/AppData/Local/Continuum/anaconda3/presto.crt'
    auth={'auth':requests.auth.HTTPBasicAuth(ppp.iloc[0]['---'], ppp.iloc[1]['---'])}
    conn = presto.connect(host="presto-sqlpy.gett.io", port=8443, protocol='https',
                          requests_kwargs=auth, requests_session=presto_session)
    data = pd.read_sql(q, conn)
    conn.close()
    return data

## Queries and processing

In [3]:
start_date = '2020-05-01'
end_date = '2020-05-31'

In [4]:
# citibank - only business rides should be in report!
# (30% discount for citibank business (10% compensation by citibank in other report))

query_rides_visa = f'''
select
fo.order_gk
,fo.sourceid
,substring(cast(fo.order_gk as varchar(20)),5,length(cast(fo.order_gk as varchar(20)))) as order_gk_str
,fo.riding_user_gk
,case
when lower(dct.class_type_desc) like '%sber%'
then 'Sberbank'
else 'VISA' end as financing_type
,fo.date_key as date_key
,case
when (lower(dct.class_type_desc_eng) like '%vip%'
     or lower(dct.class_type_desc_eng) like '%luxe%')
then 'VIP'
when lower(dct.class_type_desc_eng) like '%delivery%'
then 'Delivery'
when ((lower(dct.class_type_desc_eng) like '%moscow designated%' and fo.date_key >= date '2020-04-20')
     or (lower(dct.class_type_desc_eng) like '%sp designated%' and fo.date_key >= date '2020-04-20'))
then 'Delivery premium'     
else 'Business' end as class_group
,lower(dct.class_type_desc_eng) as class
,case
when (lower(dct.class_type_desc_eng) like '%sber%')
     or (lower(dct.class_type_desc_eng) like '%citibank%')
then fo.customer_total_cost_inc_vat/0.7
else fo.customer_total_cost_inc_vat/0.85 end as tariff
,case
when (lower(dct.class_type_desc_eng) like '%sber%')
     or (lower(dct.class_type_desc_eng) like '%citibank%')
then fo.customer_total_cost_inc_vat/0.7*0.13
else fo.customer_total_cost_inc_vat/0.85*0.13 end as visa_financing
,case
when (lower(dct.class_type_desc_eng) like '%sber%')
     or (lower(dct.class_type_desc_eng) like '%citibank%')
then fo.customer_total_cost_inc_vat/0.7*0.02
else fo.customer_total_cost_inc_vat/0.85*0.02
end as gett_financing
,fo.customer_total_cost_inc_vat as client_price
,fo.paid_with_prepaid
        
from emilia_gettdwh.dwh_fact_orders_v fo
join emilia_gettdwh.dwh_dim_class_types_v dct on dct.class_type_key = fo.class_type_key

where
fo.order_status_key in (7)
and fo.date_key between DATE '{start_date}' and DATE '{end_date}'
and fo.country_key = 2
and fo.payment_type_key = 3
and lower(dct.class_type_desc) like '%visa%'
and lower(dct.class_type_desc) not like '%citibank15%'
and lower(dct.class_type_desc) not like '%alfa%'
and lower(dct.class_type_desc) not like '%sber20%'
and lower(dct.class_type_desc) not like '%open20%'
'''

rides_visa = query_presto(query_rides_visa)

In [5]:
print(rides_visa.shape)
print(rides_visa['order_gk_str'].unique().shape)

(13020, 13)
(13020,)


In [6]:
print(rides_visa[['client_price', 'order_gk']].info())
print('----------')
print('rows with zero tariff:', rides_visa['tariff'][rides_visa['tariff'].astype('float64') == 0].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13020 entries, 0 to 13019
Data columns (total 2 columns):
client_price    12973 non-null object
order_gk        13020 non-null int64
dtypes: int64(1), object(1)
memory usage: 203.6+ KB
None
----------
rows with zero tariff: 9


## If many NaN figures do next

In [28]:
query_add = f'''
select o.id
,o.cost_total_for_company_inc_vat

from "gt-ru".gettaxi_ru_production.orders o

where o.status_id = 7
and o.created_at between (date '{start_date}' - interval '1' day) and (date '{end_date}' + interval '1' day)
'''

summ = query_presto(query_add)
print('summ.shape:', summ.shape)

summ.shape: (891656, 2)


In [60]:
os.chdir('C:/Users/maxim.buldakov/Desktop')

summ.to_csv('query_add.csv', index=False)

In [64]:
q = pd.merge(rides_visa, summ, left_on = 'sourceid', right_on = 'id', how ='left')
q = q.drop(columns = ['sourceid', 'id'])
change = ['tariff','visa_financing','gett_financing','client_price','cost_total_for_company_inc_vat']

for i in range(0, len(change)):
    q[change] = q[change].astype('float64')
    
#calculation
q['tariff2'] = q.apply(lambda x: x['cost_total_for_company_inc_vat']/0.7
                       if ('sber' in x['class'] or 'citibank' in x['class'])
                       else x['cost_total_for_company_inc_vat']/0.8 if 'business' in x['class']
                       else x['cost_total_for_company_inc_vat']/0.85, axis = 1)
q['visa_financing2'] = q.apply(lambda x: x['cost_total_for_company_inc_vat']/0.7*0.13
                               if (('sber' in x['class'] or 'citibank' in x['class']) and 'business' in x['class'])
                               else x['cost_total_for_company_inc_vat']/0.7*0.13
                               if (('sber' in x['class'] or 'citibank' in x['class']))
                               else x['cost_total_for_company_inc_vat']/0.8*0.13 if ('business' in x['class'])
                               else x['cost_total_for_company_inc_vat']/0.85*0.13, axis = 1)
q['gett_financing2'] = q.apply(lambda x: x['cost_total_for_company_inc_vat']/0.7*0.07
                               if ('sber' in x['class'] and 'business' in x['class'])
                               else x['cost_total_for_company_inc_vat']/0.7*0.02
                               if ('sber' in x['class'] or 'citibank' in x['class'])
                               else x['cost_total_for_company_inc_vat']/0.8*0.07
                               if ('business' in x['class'])
                               else x['cost_total_for_company_inc_vat']/0.85*0.02, axis = 1)

#change if NaN figures in main base
q['tariff'] = q.apply(lambda x: x['tariff2'] if (x['client_price'] == 0 or pd.isnull(x['client_price']))
                      else x['tariff'], axis = 1)
q['visa_financing'] = q.apply(lambda x: x['visa_financing2'] if (x['client_price'] == 0 or pd.isnull(x['client_price']))
                      else x['visa_financing'], axis = 1)
q['gett_financing'] = q.apply(lambda x: x['gett_financing2'] if (x['client_price'] == 0 or pd.isnull(x['client_price']))
                      else x['gett_financing'], axis = 1)
q['client_price'] = q.apply(lambda x: x['cost_total_for_company_inc_vat']
                            if (x['client_price'] == 0 or pd.isnull(x['client_price']))
                      else x['client_price'], axis = 1)
q = q.drop(columns = ['tariff2', 'visa_financing2', 'gett_financing2', 'cost_total_for_company_inc_vat'])
rides_visa = q.copy()

In [65]:
rides_visa[['client_price', 'order_gk']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50327 entries, 0 to 50326
Data columns (total 2 columns):
client_price    50327 non-null float64
order_gk        50327 non-null int64
dtypes: float64(1), int64(1)
memory usage: 1.2 MB


## Continue

In [7]:
if 'sourceid' in rides_visa.columns:
    rides_visa.drop(columns = 'sourceid', inplace = True)
    print('column sourceid deleted (no NaN figures)')
else:
    print('column sourceid not found (NaN figures part ran)')

column sourceid deleted (no NaN figures)


In [8]:
rides_visa['order_gk_str'].unique().shape

(13020,)

In [9]:
rides_visa['date_key_time'] = pd.to_datetime(rides_visa['date_key'])
rides_visa['date_key'] = rides_visa['date_key_time'].dt.date

uninque_orders = tuple(list(rides_visa['order_gk']))
uninque_orders_str = tuple(list(rides_visa['order_gk_str']))
uninque_orders_str_2 = uninque_orders_str

In [10]:
query_ftrs = f'''
with all_rides as
(select
fo.order_gk,
du.registration_date_key,
fo.date_key,
fo.ride_start_datetime as visa_ride_start_datetime,
fr.ride_start_datetime as previous_rides_start_datetime,
row_number() over (partition by fo.order_gk order by fo.order_gk, fr.ride_start_datetime ) is_ftr

from emilia_gettdwh.dwh_fact_orders_v fo
join emilia_gettdwh.dwh_fact_rides_v fr on fr.riding_user_gk = fo.riding_user_gk and fr.ride_start_datetime <= fo.ride_start_datetime
join emilia_gettdwh.dwh_dim_users_v du on du.user_gk = fo.riding_user_gk
join emilia_gettdwh.dwh_dim_class_types_v dct on dct.class_type_key = fr.class_type_key

where
fo.order_gk in {uninque_orders})

select
ar.order_gk,
case when ar.is_ftr = 1 and DATE_DIFF('day', ar.registration_date_key, ar.date_key) between 0 and 90 then 1 else 0 end as is_ftr
from all_rides ar
where ar.visa_ride_start_datetime = ar.previous_rides_start_datetime
'''

ftrs_pd = query_presto(query_ftrs)
print(ftrs_pd.shape)

(13019, 2)


In [11]:
rides_visa = pd.merge(rides_visa, ftrs_pd, left_on = 'order_gk', right_on = 'order_gk', how ='left')
print(rides_visa.shape)

(13020, 14)


In [12]:
query_tranc = f'''
select
fct.order_gk
,dcc.last4
,dcc.bin_number
,tx_type

from dwh.dim_credit_cards_v dcc
join dwh.fact_ccs_transactions_v fct on fct.credit_card_gk = dcc.credit_card_gk

where
fct.order_gk in {uninque_orders}
and fct.status = 'completed' 
and tx_type in ('charge', 'prepaid', 'settle')
and fct.exception_text is null
'''

df_trans = query_presto(query_tranc)

In [13]:
df_trans['order_gk'].unique().shape

(12959,)

## Add extra rides from redshift

In [29]:
# не запускать здесь! запрос на создание файла из redshift (DBeaver)

select
po.external_id,
pcc.last_4,
pcc.bin_number,
pt.status,
pt.tx_type
from dwh.staging_ccs.orders po
join dwh.staging_ccs.transactions pt on po.id = pt.order_id
join dwh.staging_ccs.credit_cards pcc on pt.credit_card_id = pcc.id
--join dwh.staging_ru.orders sto on po.id = sto.id
where pt.status = 'completed'
and pt.tx_type in ('charge', 'prepaid', 'settle','authorization_for_immediate_settle')
--and pt.exception_text is null
and po.env = 'RU'
and po.created_at >= '2019-06-18' and po.created_at <= '2019-07-25'
--and sto.id not in (4,9)
--and sto.payment_type_id = 3

SyntaxError: invalid syntax (<ipython-input-29-8f0c24b5ab59>, line 9)

In [19]:
os.chdir('C:/Users/maxim.buldakov/Desktop')

redshift = pd.read_csv('From_DBeaver100.csv')
redshift['external_id'] = redshift['external_id'].astype(str)
redshift['external_id'] = '2000'+redshift['external_id']
redshift['external_id'] = redshift['external_id'].astype('int64')
redshift[:3]

,external_id,last_4,bin_number,status,tx_type
0,2000951992797,2122,521324,completed,charge
1,2000948914236,2122,521324,completed,charge
2,2000955533513,3464,520935,completed,charge


In [20]:
#choose common trans with rides_visa
df_redshift = pd.merge(rides_visa, redshift, left_on = 'order_gk', right_on = 'external_id', how = 'inner')
df_redshift = df_redshift[['external_id','last_4','bin_number','tx_type']]

In [21]:
df_redshift['external_id'].unique().shape

(50192,)

In [22]:
df_redshift_extra = pd.merge(df_redshift, df_trans, left_on = 'external_id', right_on = 'order_gk', how = 'left')

In [23]:
df_redshift_extra = df_redshift_extra[df_redshift_extra['order_gk'].isnull()]
df_redshift_extra = df_redshift_extra[['external_id','last_4','bin_number_x','tx_type_x']]

In [24]:
df_redshift_extra = df_redshift_extra.rename(columns={'external_id':'order_gk',
                          'last_4':'last4',
                          'bin_number_x':'bin_number',
                          'tx_type_x':'tx_type'})
df_redshift_extra['last4'] = df_redshift_extra['last4'].astype('object')

In [25]:
df_redshift_extra

,order_gk,last4,bin_number,tx_type


In [26]:
df_redshift_extra.shape

(0, 4)

In [27]:
df_trans = df_trans.append(df_redshift_extra)

In [28]:
df_trans['order_gk'].unique().shape

(50193,)

In [29]:
df_trans.shape

(51147, 4)

In [30]:
qgrid.show_grid(df_trans, grid_options={'maxVisibleRows': 7})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Continue

In [14]:
df_trans['bin_number'] = list(map(lambda x: int(x), df_trans['bin_number']))

In [15]:
df = pd.merge(rides_visa, df_trans, left_on = 'order_gk', right_on = 'order_gk', how = 'left')
print(df.shape)

(13184, 17)


In [16]:
df = df.dropna(subset = ['bin_number'])
df['bin_number'] = list(map(lambda x: int(x),  df['bin_number']))
print(df.shape)
print(df['order_gk_str'].unique().shape)

(13123, 17)
(12959,)


In [17]:
tt = df.drop_duplicates() #RAW data

In [18]:
os.chdir('C:/Users/maxim.buldakov/Desktop/Private/VISA')

df_bin_conditions = pd.read_excel('0.visa_bin_may_2020.xlsx')
df_bin_conditions[-5:]

,BIN,CARD_TYPE,start_date,end_date
1058,467564,VISA PLATINUM,2020-04-20,2020-09-30
1059,468596,VISA INFINITE,2020-04-20,2020-09-30
1060,469670,VISA PLATINUM,2020-04-20,2020-09-30
1061,485071,VISA INFINITE,2020-04-20,2020-09-30
1062,492448,VISA SIGNATURE,2020-04-20,2020-09-30


In [19]:
df_bin_conditions['start_date'] = pd.to_datetime(df_bin_conditions['start_date'])
df_bin_conditions['end_date'] = pd.to_datetime(df_bin_conditions['end_date'])
df_fin = pd.merge(df, df_bin_conditions, left_on = 'bin_number', right_on = 'BIN', how ='inner' )
print(df_fin.shape)

(12575, 21)


In [20]:
def if_date_comp(st,en, r):
    if r <= en.date() and  r >= st.date():
        v = 1
    else:
        v = 0
    return v

df_fin['is_in_group'] = list(map(lambda st,en,r: if_date_comp(st,en,r), df_fin['start_date'] , df_fin['end_date'] , df_fin['date_key']   ) )

In [21]:
df_fin.date_key = pd.to_datetime(df_fin.date_key)
df_fin["start_date"] = pd.to_datetime(df_fin.start_date).apply(lambda x: x.date())
df_fin["end_date"] = pd.to_datetime(df_fin.end_date).apply(lambda x: x.date())
df_fin["start_date"] = pd.to_datetime(df_fin.start_date).dt.date
df_fin["end_date"] = pd.to_datetime(df_fin.end_date).dt.date
df_fin.date_key = pd.to_datetime(df_fin.date_key).dt.date

In [22]:
df_fin = df_fin[df_fin['is_in_group'] == 1]
print(df_fin.shape)
print(df_fin['order_gk_str'].unique().shape)

(12575, 22)
(12415,)


In [23]:
z = pd.concat([df_fin[df_fin.financing_type == 'VISA']
               , df_fin[df_fin.financing_type != 'VISA']], axis = 0).drop_duplicates() #.to_excel('visa_report_august_to31.xlsx')

In [24]:
z = pd.concat([df_fin[df_fin.financing_type == 'VISA']
               , df_fin[df_fin.financing_type != 'VISA']], axis = 0)
z.shape

(12575, 22)

In [25]:
z = pd.concat([df_fin[df_fin.financing_type == 'VISA']
               , df_fin[df_fin.financing_type != 'VISA']], axis = 0).drop_duplicates()
z.shape

(12573, 22)

In [26]:
x = z.copy()

## Google sheets

In [27]:
## В ЭТОМ БЛОКЕ ПОДКЛЮЧАЕМ НЕОБХОДИМЫЕ БИБЛИОТЕКИ

# Стандартные библиотеки для работы с DataFrame - так большие данные хранит Python
import pandas as pd

# Библиотеки для работы с временем
import datetime
import time

#Библиотеки для подключения к GoogleSheets
import gspread 
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd

In [28]:
import webbrowser

site1 =  "https://docs.google.com/spreadsheets/d/1zk-w4JyhPjh3pMWnh690lONzsBPFFCaVgmztrByj-Gw/edit#gid=0"
site2 =  "https://app.asana.com/0/1110565772424875/1116638790834424"
webbrowser.open(site1, new = 2)
webbrowser.open(site2, new = 2)

True

In [29]:
## В ЭТОМ БЛОКЕ ПОДКЛЮЧАЕМСЯ К GOOGLE SHEETS ТАБЛИЦЕ

# Стандартные строчки
scope = ['https://spreadsheets.google.com/feeds']

# Название файла с данными от клиента
creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/maxim.buldakov/Desktop/Private/gapi-prj1.json', scope)
client = gspread.authorize(creds)

# Ссылка на таблицу, куда подключаемся 
table = client.open_by_url(site1)

In [30]:
## В ЭТОМ БЛОКЕ ПОДКЛЮЧАЕМСЯ К ЛИСТУ В ВЫБРАННОЙ ТАБЛИЦЕ

# Подключаемся к конкретному листу в таблице
ws = table.worksheet('report')
# Очищаем этот лист
ws.clear()

{'spreadsheetId': '1zk-w4JyhPjh3pMWnh690lONzsBPFFCaVgmztrByj-Gw',
 'clearedRange': 'report!A1:T62316'}

In [31]:
#add part
x1 = x.copy()
x1 = x1.drop(['order_gk', 'riding_user_gk', 'date_key_time', 'start_date', 'end_date', 'is_in_group'], axis = 1)
x1['paid_with_prepaid'] = x1['paid_with_prepaid'].astype('float64')
x1['paid_with_prepaid'] = x1['paid_with_prepaid'].fillna(0)
x1['paid_with_prepaid'] = x1['paid_with_prepaid'].apply(lambda row: 1 if row > 0 else 0)
x1 = x1.sort_values(by = ['order_gk_str', 'tx_type'])
gd.set_with_dataframe(ws, x1, row=1, col=1)

### Card type by hand

In [41]:
text = f'''
select lpm.*, ms.credit_card_id

from bp_ba.latest_payment_methods lpm
join bp_ba.money_spent_from_card ms on lpm.fingerprint = ms.fingerprint
'''

cards_info = query_presto(text)
print(cards_info.shape)

(4129443, 22)


In [98]:
cards = cards_info.copy()
print(cards.shape)
cards.dropna(subset=['user_gk', 'bin_number', 'last_4_digits'], inplace=True)
print(cards.shape)
cards = cards[cards['bin_number'] != 'nan']
print(cards.shape)

(4129443, 22)
(4129443, 22)
(4129396, 22)


In [99]:
cards['user_gk'] = cards['user_gk'].astype('int64')
cards['bin_number'] = cards['bin_number'].astype('int64')

In [100]:
tt_add = pd.merge(tt, cards
                  ,how = 'left'                 
                  ,left_on=['riding_user_gk', 'bin_number', 'last4']
                 ,right_on=['user_gk', 'bin_number', 'last_4_digits'])

In [109]:
var = tt_add
text = f'''tt_add'''

os.chdir('C:/Users/maxim.buldakov/Desktop/Temp_python')
var.to_excel(text + '.xlsx', index = True)
subprocess.Popen('C:/Users/maxim.buldakov/Desktop/Temp_python/' + text + '.xlsx', shell=True)

### End of part card type by hand

In [32]:
dates = {'From': [start_date], 'To': [end_date]}
dates = pd.DataFrame(data=dates)

In [33]:
summary = table.worksheet('summary')
#summary.clear()

In [34]:
gd.set_with_dataframe(summary, dates ,row=1, col=1)

In [35]:
sber = x[(x['financing_type'] =='Sberbank')&((x.BIN ==427417)| (x.BIN==483983)) ]

In [36]:
sberbank  = table.worksheet('sberbank raw')
sberbank.clear()

{'spreadsheetId': '1zk-w4JyhPjh3pMWnh690lONzsBPFFCaVgmztrByj-Gw',
 'clearedRange': "'sberbank raw'!A1:Z7438"}

In [37]:
gd.set_with_dataframe(sberbank, sber ,row=1, col=1)

In [38]:
# Подключаемся к конкретному листу в таблице
ws1 = table.worksheet('raw data')
# Очищаем этот лист
ws1.clear()

{'spreadsheetId': '1zk-w4JyhPjh3pMWnh690lONzsBPFFCaVgmztrByj-Gw',
 'clearedRange': "'raw data'!A1:Z62252"}

In [39]:
gd.set_with_dataframe(ws1, tt, row=1, col=1)

In [40]:
#end of script